In [157]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import datetime


In [158]:
%matplotlib inline

In [159]:
%config InlineBackend.figure_format = 'retina'

In [160]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [161]:
%autoreload 1

In [162]:
%aimport analysis

In [163]:
conditions = pd.read_csv("conditions.csv")
patients = pd.read_csv("patients.csv")
observations = pd.read_csv("observations.csv")
care_plans = pd.read_csv("careplans.csv")
encounters = pd.read_csv("encounters.csv")
devices = pd.read_csv("devices.csv")
supplies = pd.read_csv('supplies.csv')
procedures = pd.read_csv("procedures.csv")
medications = pd.read_csv("medications.csv")

KeyboardInterrupt: 

In [134]:
conditions.head(3)

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2019-02-15,2019-08-01,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,d5ee30a9-362f-429e-a87a-ee38d999b0a5,65363002,Otitis media
1,2019-10-30,2020-01-30,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,8bca6d8a-ab80-4cbf-8abb-46654235f227,65363002,Otitis media
2,2020-03-01,2020-03-30,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,681c380b-3c84-4c55-80a6-db3d9ea12fee,386661006,Fever (finding)


In [135]:
covid_patient_ids = conditions[conditions.CODE == 840539006].PATIENT.unique()

In [136]:
len(covid_patient_ids)

8820

In [137]:
deceased_patients = patients[patients.DEATHDATE.notna()].Id

In [138]:
len(deceased_patients)

2352

In [139]:
deceased_patient_ids = deceased_patients.array.to_numpy()

Find COVID-19 patients that have not died yet

In [140]:
alive_covid_patients_id = np.setdiff1d(covid_patient_ids, deceased_patient_ids)

In [141]:
conditions = conditions[conditions.PATIENT.isin(alive_covid_patients_id)]
patients = patients[patients.Id.isin(alive_covid_patients_id)]
observations = observations[observations.PATIENT.isin(alive_covid_patients_id)]
care_plans = care_plans[care_plans.PATIENT.isin(alive_covid_patients_id)]
encounters = encounters[encounters.PATIENT.isin(alive_covid_patients_id)]
devices = devices[devices.PATIENT.isin(alive_covid_patients_id)]
supplies = supplies[supplies.PATIENT.isin(alive_covid_patients_id)]
procedures = procedures[procedures.PATIENT.isin(alive_covid_patients_id)]
medications = medications[medications.PATIENT.isin(alive_covid_patients_id)]

In [142]:
conditions.to_csv('filtered_conditions.csv', index=False)
patients.to_csv('filtered_patients.csv', index=False)
observations.to_csv('filtered_observations.csv', index=False)
care_plans.to_csv('filtered_care_plans.csv', index=False)
encounters.to_csv('filtered_encounters.csv', index=False)
devices.to_csv('filtered_devices.csv', index=False)
supplies.to_csv('filtered_supplies.csv', index=False)
procedures.to_csv('filtered_procedures.csv', index=False)
medications.to_csv('filtered_medications.csv', index=False)


In [143]:
import category_encoders as ce
print(conditions.head(1))
conditionEncoder = ce.OneHotEncoder(cols=["DESCRIPTION"], use_cat_names=True)

START        STOP                               PATIENT  \
0  2019-02-15  2019-08-01  f0f3bc8d-ef38-49ce-a2bd-dfdda982b271   

                              ENCOUNTER      CODE   DESCRIPTION  
0  d5ee30a9-362f-429e-a87a-ee38d999b0a5  65363002  Otitis media  


In [144]:
encodedConditions = conditionEncoder.fit_transform(conditions['DESCRIPTION'])

In [145]:
conditions = pd.concat([conditions, encodedConditions], axis=1)

In [146]:
aggregated_conditions = conditions.groupby('PATIENT').agg(sum)
aggregated_conditions = aggregated_conditions.drop(columns=['CODE'])
aggregated_conditions.head(2)

,DESCRIPTION_Otitis media,DESCRIPTION_Fever (finding),DESCRIPTION_Suspected COVID-19,DESCRIPTION_COVID-19,DESCRIPTION_Sprain of ankle,DESCRIPTION_Cough (finding),DESCRIPTION_Sputum finding (finding),DESCRIPTION_Diarrhea symptom (finding),DESCRIPTION_Streptococcal sore throat (disorder),DESCRIPTION_Hypertension,...,DESCRIPTION_Non-small cell carcinoma of lung TNM stage 1 (disorder),DESCRIPTION_Blindness due to type 2 diabetes mellitus (disorder),DESCRIPTION_Male Infertility,DESCRIPTION_Cystic Fibrosis,DESCRIPTION_History of amputation of foot (situation),DESCRIPTION_Third degree burn,DESCRIPTION_Acute Cholecystitis,DESCRIPTION_Cholelithiasis,DESCRIPTION_Traumatic brain injury (disorder),DESCRIPTION_Injury of kidney (disorder)
PATIENT,,,,,,,,,,,,,,,,,,,,,
0000b247-1def-417a-a783-41c8682be022,0,1,1,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00049ee8-5953-4edd-a277-b9c1b1a7f16b,0,1,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Encode patient data

In [147]:
now = pd.to_datetime('now')
patients['AGE'] = ((now - pd.to_datetime(patients['BIRTHDATE'])).astype('<m8[Y]'))
patients.head(2)

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,AGE
0,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,2017-08-24,NaN,999-68-6630,NaN,NaN,NaN,Jacinto644,Kris249,NaN,...,888 Hickle Ferry Suite 38,Springfield,Massachusetts,Hampden County,1106.0,42.151961,-72.598959,8446.49,1499.08,3.0
1,067318a4-db8f-447f-8b6e-f2f61e9baaa5,2016-08-01,NaN,999-15-5895,NaN,NaN,NaN,Alva958,Krajcik437,NaN,...,1048 Skiles Trailer,Walpole,Massachusetts,Norfolk County,2081.0,42.177370,-71.281353,89893.40,1845.72,4.0


In [148]:
patientRaceEncoder = ce.OneHotEncoder(cols=["RACE"], use_cat_names=True)
encodedPatients = patientRaceEncoder.fit_transform(patients['RACE'])
encodedPatients.head(2)

,RACE_white,RACE_native,RACE_asian,RACE_black,RACE_other
0,1,0,0,0,0
1,1,0,0,0,0


In [149]:
patients = pd.concat([patients, encodedPatients], axis=1)

In [150]:
patients.head(2)

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,AGE,RACE_white,RACE_native,RACE_asian,RACE_black,RACE_other
0,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,2017-08-24,NaN,999-68-6630,NaN,NaN,NaN,Jacinto644,Kris249,NaN,...,42.151961,-72.598959,8446.49,1499.08,3.0,1,0,0,0,0
1,067318a4-db8f-447f-8b6e-f2f61e9baaa5,2016-08-01,NaN,999-15-5895,NaN,NaN,NaN,Alva958,Krajcik437,NaN,...,42.177370,-71.281353,89893.40,1845.72,4.0,1,0,0,0,0


Merge Conditions and Patient details

In [164]:
patient_age = patients.drop(patients.columns.difference(['Id', 'AGE']), axis=1)
patient_age.head(2)

,Id
0,f0f3bc8d-ef38-49ce-a2bd-dfdda982b271
1,067318a4-db8f-447f-8b6e-f2f61e9baaa5


In [152]:
obsVals = observations[observations.TYPE != 'text']
obsrvEncoder = ce.OneHotEncoder(cols=["DESCRIPTION"], use_cat_names=True)
encodedObs = obsrvEncoder.fit_transform(test['DESCRIPTION'])
encodedObs.head(2)

,DESCRIPTION_Body Height,DESCRIPTION_Pain severity - 0-10 verbal numeric rating [Score] - Reported,DESCRIPTION_Body Weight,DESCRIPTION_Weight-for-length Per age and sex,DESCRIPTION_Head Occipital-frontal circumference,DESCRIPTION_Diastolic Blood Pressure,DESCRIPTION_Systolic Blood Pressure,DESCRIPTION_Heart rate,DESCRIPTION_Respiratory rate,DESCRIPTION_Body Mass Index,...,DESCRIPTION_ALT (Elevated),DESCRIPTION_AST (Elevated),DESCRIPTION_Anion Gap,DESCRIPTION_Protein,DESCRIPTION_Albumin,DESCRIPTION_Globulin,DESCRIPTION_Alkaline Phosphatase,DESCRIPTION_Lymph nodes with micrometastases [#] in Cancer specimen by Light microscopy,DESCRIPTION_PROMIS-10 Global Mental Health (GMH) score,DESCRIPTION_PROMIS-10 Global Physical Health (GPH) score
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [153]:
obsVals = pd.concat([obsVals, encodedObs], axis=1)


In [156]:
obsVals.head(2)

NameError: name 'obsVals' is not defined